In [3]:
"""docstring :)"""

'docstring :)'

In [33]:
import json
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
# from time import sleep

with open('data/api-key.txt', 'r') as api:
    API_KEY = api.read()
lol_watcher = LolWatcher(API_KEY)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4)"+
                    " AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135"+
                    " Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

CHAMP_ID = 64
TESTMATCHID = "4748107995"

np.random.seed(7)

matches = pd.read_csv("data/clid-matches.csv")

the time stamp 44111 is 44 seconds and 111 milliseconds

In [5]:
matches

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4768709088,64,420,13,1604760613378,NONE,JUNGLE
1,KR,4766414121,64,420,13,1604685379075,NONE,JUNGLE
2,KR,4766279291,64,420,13,1604682892465,NONE,JUNGLE
3,KR,4766343350,64,420,13,1604680882287,NONE,JUNGLE
4,KR,4764140809,64,420,13,1604585076015,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
523,KR,3434837338,64,420,12,1543122169501,NONE,JUNGLE
524,KR,3432455664,64,420,12,1542973849243,NONE,JUNGLE
525,KR,3430594315,64,420,12,1542822782724,NONE,JUNGLE
526,KR,3420476831,64,420,12,1542113321062,NONE,JUNGLE


In [115]:
a = [1]
b = []
b.append(a)
a.append(b)
b == b[0][1]

True

In [34]:
def get_match(match_id, region="kr"):
    """matchid is str, returns:
    match_json is a json obj that has whole match's data
    player_participant_id is clid's id in this game
    blue_team is the set of ids on the blue team not counting clid if clid is on blue
    red_team is same as blue but for red team
    the team that has clid on it is returned first.
    """
    match_request = requests.get("https://"+region+".api.riotgames.com/lol/match/v4/matches/"+match_id,
                                 headers=HEADERS)
    match_json = json.loads(match_request.text)
    
    blue_team = []
    red_team = []
    for participant in match_json["participants"]:
        if participant["championId"] == CHAMP_ID:
            player_participant_id = participant["participantId"]
        elif participant["teamId"] == 100:
            blue_team.append(participant["participantId"])
        else:
            red_team.append(participant["participantId"])
    if len(red_team) == 4:
        return match_json, player_participant_id, red_team, blue_team
    else:
        return match_json, player_participant_id, blue_team, red_team

In [180]:
def get_timeline(match_id, ppi, ally_team_ids, enemy_team_ids, region="kr", give_json=True):
    url = ["https://", ".api.riotgames.com/lol/match/v4/timelines/by-match/"]
    url = "".join([url[0], region, url[1], match_id])
    timeline_res = requests.get(url, headers=HEADERS)

    timeline_obj = json.loads(timeline_res.text)
    
    ally_frames = []
    enemy_frames = []
    player_frames = []
    for frame in timeline_obj["frames"]:
        timestamp = frame["timestamp"]
        for value in frame["participantFrames"].values():
            try:
                value["x"] = value["position"]["x"]
                value["y"] = value["position"]["y"]
            except:
                value["x"] = np.nan
                value["y"] = np.nan

        frame_df = pd.DataFrame.from_dict(frame["participantFrames"])#.values.flatten()
        frame_df = frame_df.drop(["position", "teamScore", "dominionScore", "xp", "minionsKilled"], errors="ignore")
        frame_df = frame_df.T.set_index("participantId")
        
        player_series = frame_df.loc[ppi]
        ally_df = frame_df.loc[ally_team_ids]
        enemy_df = frame_df.loc[enemy_team_ids]
        
        flatten_ally = ally_df.values.flatten()
        flatten_enemy = enemy_df.values.flatten()
        
        player_frames.append(player_series)
        ally_frames.append(flatten_ally)
        enemy_frames.append(flatten_enemy)
        
    
    ally_cols = ["".join(["ally"+str(i), col]) for i in range(1,5) for col in frame_df.columns]
    enemy_cols = ["".join(["enemy"+str(i), col]) for i in range(6,11) for col in frame_df.columns]
    player_cols = ["".join(["player",col]) for col in frame_df.columns]
    
    player_df = pd.concat(player_frames,axis=1).T.reset_index(drop=True)
    player_df.columns = player_cols
    ally_df = pd.DataFrame(ally_frames, columns=ally_cols)
    enemy_df = pd.DataFrame(enemy_frames, columns=enemy_cols)
    
    full_df = pd.concat([enemy_df, ally_df, player_df],axis=1)
    if give_json:
        return {"df": full_df, "json": timeline_obj}
    else:
        return {"df": full_df}
    

In [177]:
# test
#match, ppi, allyteam, enemyteam = get_match(TESTMATCHID)
#ppi, allyteam, enemyteam

In [178]:
ppi, allyteam, enemyteam

(1, [2, 3, 4, 5], [6, 7, 8, 9, 10])

In [179]:
get_timeline(TESTMATCHID, ppi, allyteam, enemyteam)

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

,enemy6currentGold,enemy6totalGold,enemy6level,enemy6jungleMinionsKilled,enemy6x,enemy6y,enemy7currentGold,enemy7totalGold,enemy7level,enemy7jungleMinionsKilled,...,ally4level,ally4jungleMinionsKilled,ally4x,ally4y,playercurrentGold,playertotalGold,playerlevel,playerjungleMinionsKilled,playerx,playery
0,500.0,500.0,1.0,0.0,14486.0,14291.0,500.0,500.0,1.0,0.0,...,1.0,0.0,560.0,581.0,500,500,1,0,560,361
1,0.0,500.0,1.0,0.0,7060.0,9588.0,0.0,500.0,1.0,0.0,...,1.0,0.0,2888.0,11098.0,0,500,1,0,4580,6885
2,121.0,621.0,2.0,4.0,11984.0,6891.0,65.0,565.0,1.0,0.0,...,1.0,0.0,1836.0,12199.0,121,621,2,4,2639,8080
3,518.0,1018.0,3.0,16.0,6634.0,11506.0,259.0,759.0,2.0,0.0,...,3.0,0.0,2853.0,13321.0,518,1018,3,16,7145,3739
4,1373.0,1873.0,4.0,28.0,6431.0,8093.0,394.0,894.0,3.0,0.0,...,4.0,0.0,1507.0,11145.0,101,1376,4,24,5437,4947
5,116.0,2041.0,5.0,29.0,6989.0,9945.0,584.0,1084.0,3.0,0.0,...,5.0,0.0,3153.0,13180.0,620,1895,4,28,4137,13003
6,1217.0,3142.0,6.0,36.0,8106.0,10498.0,785.0,1285.0,4.0,0.0,...,5.0,0.0,1534.0,8762.0,163,2338,4,28,2426,8373
7,1529.0,3454.0,6.0,44.0,10060.0,4433.0,274.0,1449.0,4.0,0.0,...,6.0,0.0,4117.0,9828.0,560,2735,5,40,5570,7588
8,234.0,3734.0,7.0,48.0,5651.0,12879.0,58.0,1673.0,5.0,0.0,...,7.0,0.0,1701.0,12879.0,857,3032,6,47,3365,8034
9,530.0,4030.0,7.0,54.0,7649.0,11545.0,193.0,1883.0,5.0,0.0,...,8.0,0.0,2374.0,9869.0,48,3523,6,55,2484,1600


In [183]:
def get_n_timelines(match_df, start_index, end_index, give_json=True, region="kr"):
    matches = {}
    dfs = {}
    if give_json:
        jsons = {}
    for index in match_df.index[start_index:end_index]:
        current_id = str(match_df.loc[index]["gameId"])
        print(current_id)
        matches[current_id], ppi, ally_team, enemy_team = get_match(current_id)
        result = get_timeline(current_id, ppi, ally_team, enemy_team, region, give_json)
        dfs[current_id] = result["df"]
        if give_json:
            jsons[current_id] = result["json"]
    if give_json:
        return matches, dfs, jsons
    else:
        return matches, dfs

fetching timelines - I don't have a function to make the calls fit the api limits so you have to pause briefly between each of the following cells - its not bad though.

In [184]:
result1 = get_n_timelines(matches, 0, 50)

4768709088
['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlev

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

In [185]:
result2 = get_n_timelines(matches, 50, 100)

4542083040
['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlev

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

In [188]:
#import time
#time.sleep(20)
result3 = get_n_timelines(matches, 100, 150)

4425852270
['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlev

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

In [198]:
result4 = get_n_timelines(matches, 150, 200)

4174345259
['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlev

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

['ally1currentGold', 'ally1totalGold', 'ally1level', 'ally1jungleMinionsKilled', 'ally1x', 'ally1y', 'ally2currentGold', 'ally2totalGold', 'ally2level', 'ally2jungleMinionsKilled', 'ally2x', 'ally2y', 'ally3currentGold', 'ally3totalGold', 'ally3level', 'ally3jungleMinionsKilled', 'ally3x', 'ally3y', 'ally4currentGold', 'ally4totalGold', 'ally4level', 'ally4jungleMinionsKilled', 'ally4x', 'ally4y'] ['enemy6currentGold', 'enemy6totalGold', 'enemy6level', 'enemy6jungleMinionsKilled', 'enemy6x', 'enemy6y', 'enemy7currentGold', 'enemy7totalGold', 'enemy7level', 'enemy7jungleMinionsKilled', 'enemy7x', 'enemy7y', 'enemy8currentGold', 'enemy8totalGold', 'enemy8level', 'enemy8jungleMinionsKilled', 'enemy8x', 'enemy8y', 'enemy9currentGold', 'enemy9totalGold', 'enemy9level', 'enemy9jungleMinionsKilled', 'enemy9x', 'enemy9y', 'enemy10currentGold', 'enemy10totalGold', 'enemy10level', 'enemy10jungleMinionsKilled', 'enemy10x', 'enemy10y'] ['playercurrentGold', 'playertotalGold', 'playerlevel', 'playe

In [201]:
# rdd = result_df_dis 
rdd = [result1[1],result2[1],result3[1], result4[1]]
result_ids = set(rdd[0].keys())
result_ids.update(result2[1].keys(),result3[1].keys(),result4[1].keys())#
len(result_ids)

200

In [202]:
dfs_dict = {**rdd[0], **rdd[1], **rdd[2], **rdd[3]}

In [203]:
len(dfs_dict.keys())

200

In [204]:
import pickle
pickle.dump(dfs_dict, open("data/timeline-di.p", "wb"))

In [205]:
rjd = [result1[2],result2[2],result3[2]]#,result4[2]]
jsons_dict = {**rjd[0], **rjd[1], **rjd[2]}#, **rjd[3]}

In [206]:
pickle.dump(jsons_dict, open("data/json-timeline-di.p", "wb"))

In [207]:
rmd = [result1[0],result2[0],result3[0]]#,result4[0]]
match_dict = {**rmd[0], **rmd[1], **rmd[2]}#, **rmd[3]}

In [208]:
pickle.dump(match_dict, open("data/match-di.p", "wb"))